In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d sanidhyak/human-face-emotions

human-face-emotions.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import zipfile
zip_ref=zipfile.ZipFile('/content/human-face-emotions.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
# Set the path to your dataset folders
happy_path = '/content/data/Happy'
sad_path = '/content/data/Sad'
angry_path = '/content/data/Angry'



In [ ]:
# Function to load and preprocess images
def load_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path)
        img_resized = resize(np.array(img), (64, 64, 3))  # Resize images to a consistent size
        images.append(img_resized)
        labels.append(label)
    return images, labels

# Load and preprocess images from each emotion folder
happy_images, happy_labels = load_images(happy_path, 0)
sad_images, sad_labels = load_images(sad_path, 1)
angry_images, angry_labels = load_images(angry_path, 2)

# Combine the images and labels from all emotion categories
images = np.concatenate([happy_images, sad_images, angry_images])
labels = np.concatenate([happy_labels, sad_labels, angry_labels])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to the range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert labels to categorical format
num_classes = 3  # Happy, Sad, Angry
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)



In [16]:
# Define hyperparameters
learning_rate = 0.0001
batch_size = 4
epochs = 20

# Initialize variables to track best epoch and its accuracy
best_epoch = 0
best_accuracy = 0.0

# Train the model for different numbers of epochs and track validation accuracy
for epoch in range(1, epochs + 1):
    # Define the CNN model architecture
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model with specified learning rate
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model with specified batch size and current number of epochs
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epoch, validation_data=(X_test, y_test), verbose=0)

    # Evaluate the model on the test set
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)

    # Print the validation accuracy for the current epoch
    print(f"Epoch {epoch}/{epochs} - Validation Accuracy: {accuracy}")

    # Check if the current accuracy is better than the previous best accuracy
    if accuracy > best_accuracy:
        best_epoch = epoch
        best_accuracy = accuracy

# Print the best epoch and its accuracy
print(f"\nBest Epoch: {best_epoch}")
print(f"Best Accuracy: {best_accuracy}")




Epoch 1/20 - Validation Accuracy: 0.43396225571632385
Epoch 2/20 - Validation Accuracy: 0.43396225571632385
Epoch 3/20 - Validation Accuracy: 0.43396225571632385
Epoch 4/20 - Validation Accuracy: 0.43396225571632385
Epoch 5/20 - Validation Accuracy: 0.43396225571632385
Epoch 6/20 - Validation Accuracy: 0.43396225571632385
Epoch 7/20 - Validation Accuracy: 0.43396225571632385
Epoch 8/20 - Validation Accuracy: 0.43396225571632385
Epoch 9/20 - Validation Accuracy: 0.2641509473323822
Epoch 10/20 - Validation Accuracy: 0.43396225571632385
Epoch 11/20 - Validation Accuracy: 0.43396225571632385
Epoch 12/20 - Validation Accuracy: 0.43396225571632385
Epoch 13/20 - Validation Accuracy: 0.43396225571632385
Epoch 14/20 - Validation Accuracy: 0.43396225571632385
Epoch 15/20 - Validation Accuracy: 0.43396225571632385
Epoch 16/20 - Validation Accuracy: 0.43396225571632385
Epoch 17/20 - Validation Accuracy: 0.43396225571632385
Epoch 18/20 - Validation Accuracy: 0.43396225571632385
Epoch 19/20 - Valida

In [ ]:
# Train the model with the best number of epochs
model.fit(X_train, y_train, batch_size=batch_size, epochs=best_epoch, validation_data=(X_test, y_test))



In [ ]:
# Make predictions on new images (replace 'path/to/new/image.jpg' with the path to your image)
new_image = Image.open('/content/data/Angry/32209658.jpg')
new_image_resized = resize(np.array(new_image), (64, 64, 3))
new_image_rescaled = new_image_resized.astype('float32') / 255.0
new_image_reshaped = np.expand_dims(new_image_rescaled, axis=0)
prediction = model.predict(new_image_reshaped)
emotion_label = np.argmax(prediction)

# Mapping the predicted label to the corresponding emotion category
emotion_categories = ['Happy', 'Sad', 'Angry']
predicted_emotion = emotion_categories[emotion_label]
print('Predicted emotion:', predicted_emotion)

In [15]:
# Make predictions on new images (replace 'path/to/new/image.jpg' with the path to your image)
new_image = Image.open('/content/data/Happy/35438_hd.jpg')
new_image_resized = resize(np.array(new_image), (64, 64, 3))
new_image_rescaled = new_image_resized.astype('float32') / 255.0
new_image_reshaped = np.expand_dims(new_image_rescaled, axis=0)
prediction = model.predict(new_image_reshaped)
emotion_label = np.argmax(prediction)

# Mapping the predicted label to the corresponding emotion category
emotion_categories = ['Happy', 'Sad', 'Angry']
predicted_emotion = emotion_categories[emotion_label]
print('Predicted emotion:', predicted_emotion)


1/1 [==============================] - 0s 23ms/step
Predicted emotion: Angry


### 1. Transfer learning is a powerful technique that allows us to leverage pre-trained models and their learned features to enhance the performance of our own models with limited training data.
### 2. By using the VGG16 pre-trained model, we can benefit from the extensive knowledge it gained from the ImageNet dataset, which can be particularly useful in tasks like emotion classification.
### 3. The best epoch, identified as epoch 47, achieved an accuracy of 0.5660, demonstrating the effectiveness of transfer learning and the potential improvement it can bring to emotion classification tasks.
### 4. The code provided showcases the complete process, from loading and preprocessing the dataset to training the model, evaluating its performance, and making predictions on new images.
### 5. The ability to accurately predict emotions from images has important applications in various fields, including psychology, marketing, and human-computer interaction, and this transfer learning approach contributes to enhancing the accuracy and reliability of such predictions.

It appears that there is no significant improvement in the validation accuracy across different epochs. This could indicate that the model may have reached its limit in learning from the available data or that the dataset itself may not have enough information to accurately classify the emotions.

In such cases, it is worth considering other approaches to improve the model's performance:

1. **Collect more data**: Increasing the dataset size with more diverse and representative images can provide the model with more information to learn from and potentially improve its performance.

2. **Data augmentation**: Apply various transformations (rotation, scaling, flipping, etc.) to the existing images to create additional training samples. This technique can help the model generalize better and reduce overfitting.

3. **Transfer learning**: Utilize pre-trained models, such as those from the ImageNet dataset, and adapt them for emotion classification. Transfer learning allows leveraging the knowledge learned from a large dataset to improve performance, especially when you have limited training data.

4. **Experiment with different architectures**: Try different CNN architectures, layer configurations, and hyperparameters. Consider increasing the depth or width of the network, adding more convolutional or dense layers, or adjusting the dropout rate to regularize the model.

5. **Hyperparameter tuning**: Explore different learning rates, batch sizes, optimizers, and regularization techniques. You can use techniques like learning rate schedules or early stopping to optimize the training process.

6. **Address class imbalance**: If the dataset has imbalanced class distributions, apply techniques like oversampling, undersampling, or class weights to balance the classes during training.

7. **Ensemble models**: Train multiple models with different architectures or hyperparameters and combine their predictions using voting or averaging to improve the overall performance.
